In [2]:
!npm i -g localtunnel
!pip install bs4
!pip install py_olamaps
!pip install openai


m#################.] / reify:yargs-parser: http fetch GET 200 https://registry.
added 22 packages in 2s

3 packages are looking for funding
  run `npm fund` for details
npm notice 
npm notice New minor version of npm available! 10.5.0 -> 10.8.3
npm notice Changelog: https://github.com/npm/cli/releases/tag/v10.8.3
npm notice Run npm install -g npm@10.8.3 to update!
npm notice 
  Attempting uninstall: retrying
    Found existing installation: retrying 1.3.3
    Uninstalling retrying-1.3.3:
      Successfully uninstalled retrying-1.3.3


In [5]:
from flask import request as flask_request, Flask,jsonify
import requests
import urllib
from requests.structures import CaseInsensitiveDict

from pprint import pprint
from bs4 import BeautifulSoup
import multiprocessing
import time
from dotenv import load_dotenv
import os
import threading
import pandas as pd
import numpy as np
import os
from py_olamaps.OlaMaps import OlaMaps
from openai import OpenAI

In [4]:
load_dotenv('.env')
CLIENT_ID = os.getenv('MAPPLS_CLIENT_ID')
CLIENT_SECRET = os.getenv('MAPPLS_CLIENT_SECRET')


In [6]:
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "image_url",
          "image_url": {
            "url": "https://cdn.collectorbazar.com/products/india-postal-envelope-registered-letter-commerially-used-g18657-268009-1.jpg"
          }
        },
        {
          "type": "text",
          "text": "The following image is of a letter , you need accurately identify only the reciever's address and return it to me . Your answer should consist of nothing but the reciever's address. If the address is not the reciever's address simply return not found"
        }
      ]
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  response_format={
    "type": "text"
  }
)

In [14]:
def get_parse_address_from_image(self,url):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": url,
                    }
                    },
                    {
                        "type": "text",
                        "text": "The following image is of a letter , you need accurately identify only the reciever's address and return it to me . Your answer should consist of nothing but the reciever's address. If the address is not the reciever's address simply return not found"
                    }
                ]
            }
        ],
        temperature=1,
        max_tokens=256,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={
            "type": "text"
        }
    )
    a=str(response.choices[0].message.content)
    return ', '.join(a.split('\n'))

In [22]:
get_parse_address_from_image('_',"https://cdn.collectorbazar.com/products/india-postal-envelope-registered-letter-commerially-used-g18657-268009-1.jpg")

'Hindustan Lever Ltd., Express Building 1st Floor, Bahadur Shah Zafer Marg, P.O. Box. 7003,, N E W  D E L H I - 110002'

In [13]:
a=str(response.choices[0].message.content)
', '.join(a.split('\n'))

'Hindustan Lever Ltd., Express Building 1st Floor, Bahadur Shah Zafer Marg, P.O. Box. 7003,, NEW DELHI-110002'

In [17]:
class MMI:
    def __init__(self, client_id, client_secret):
        self.authorization_header = self.get_authorization_header(client_secret=client_secret,client_id=client_id)
        self.client_id = client_id
        self.client_secret = client_secret
        self.df : pd.DataFrame = pd.read_pickle('pincode.pkl')
        self.client = OlaMaps(
            api_key=os.environ.get("OLA_MAPS_API_KEY"),
        )

    def get_geo_coded_data(self,address):
        encoded_address = address
        
        url = f'https://atlas.mappls.com/api/places/geocode?region=ind&address={encoded_address}&itemCount=1&bias=0'
        retry_authentication = True
        while True:
            response = requests.get(url, headers=self.authorization_header)
            if response.status_code == 200:
                return {
                    'status' : 'success',
                    'status_code' : response.status_code,
                    'response' : response.json()['copResults']
                }
            elif response.status_code == 204:
                return {
                    'status' : 'Address not found',
                    'status_code' : response.status_code,
                    'response' : response
                }
            elif response.status_code == 401:
                if retry_authentication:
                    print("Retrying and refreshing the token")
                    self.authorization_header = self.get_authorization_header(self.client_id,self.client_secret)
                else: 
                    return {
                        'status' : 'Authorization Fail',
                        'status_code' : response.status_code,
                        'response' : response
                    }
                    
            elif response.status_code >= 400:
                return {
                    'status' : 'Authorization Fail or Server Error',
                    'status_code' : response.status_code,
                    'response' : response
                }
            else:
                return {
                    'status' : 'Some Other Error',
                    'status_code' : response.status_code,
                    'response' : response
                }
            
    def get_authorization_key(self,client_id, client_secret):
        response = requests.post(
            'https://outpost.mappls.com/api/security/oauth/token',
            data={
                'grant_type':'client_credentials',
                'client_id':client_id,
                'client_secret':client_secret 
            }
        )
        resp_json=response.json()
        return f"{resp_json.get('token_type')} {resp_json.get('access_token')}"
    
    def get_authorization_header(self,client_id, client_secret):
        return {
            'Authorization' : self.get_authorization_key(client_id=client_id,client_secret=client_secret)
        }  
        
    def get_coordinates_from_eLoc(self,eloc):
        headers = headers = {
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
            "Accept-Encoding": "gzip, deflate, br, zstd",
            "Accept-Language": "en-US,en;q=0.9",
            "Cache-Control": "max-age=0",
            "Cookie": "_d3=eff5c0e04d082d2886173764eb5c26a3; _d4=eff5c0e04d082d2886173764eb5c26a3; PHPSESSID=c8i2en9qf2cbjv2mijrfuf2qa8; _autologin=aXNoYbmpW4xMzA4MDNfMTcyNTE3NjE4My05NGRhYTcyNDY5OGViMjI3LTEyOC4wLjAuMC1DaHJvbWUtNDQyOWFkZDYzNWZjMWEzODRmODY1MGQwYzFmZmU4NzU%3D",
            "Priority": "u=0, i",
            "Sec-CH-UA": '"Chromium";v="128", "Not;A=Brand";v="24", "Google Chrome";v="128"',
            "Sec-CH-UA-Mobile": "?1",
            "Sec-CH-UA-Platform": '"Android"',
            "Sec-Fetch-Dest": "document",
            "Sec-Fetch-Mode": "navigate",
            "Sec-Fetch-Site": "none",
            "Sec-Fetch-User": "?1",
            "Upgrade-Insecure-Requests": "1",
            "User-Agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Mobile Safari/537.36"
        }
        response = requests.get(f'https://www.mappls.com/{eloc}', headers=headers)
        if response.ok:
            soup = BeautifulSoup(response.content, 'html.parser')
            latitude = soup.find('input', {'id': 'lati'})
            longitude = soup.find('input', {'id' : 'longi'})
            d = {}
            if latitude and longitude:
                d['long'] = latitude.get('value')
                d['lat'] = longitude.get('value')
                return d
            else:
                raise Exception(f"Unable to retrieve the coordinates status_code : {response.status_code}")
        
            return response.json()
        else:
            raise Exception("Response is not ok")

    def get_geo_coded_data_with_coordinates(self,address):
        api_response = self.get_geo_coded_data(address=address)
        if api_response.get('status') != 'success':
            return api_response
        
        try:
            eloc = api_response['response']['eLoc']
            coordinate_data = self.get_coordinates_from_eLoc(eloc=eloc)
            api_response['response'].update(coordinate_data)
            return api_response
        except Exception as e:
            api_response['status'] = 'failed'
            api_response['exception'] = e
            return api_response
        
    def find_nearest_postoffices_distances(self,lat,long,limit = 5):
        lat = np.float64(lat)
        long = np.float64(long)
        target_coord = (lat,long)
        distance_2 = (self.df['Latitude'].to_numpy() - target_coord[0])**2 + (self.df['Longitude'].to_numpy() - target_coord[1])**2
        ser = pd.Series(np.sqrt(distance_2.astype(np.float64)))
        return ser.sort_values().head(limit)
        
    def get_nearest_postoffice(self,lat,long):
        lat = np.float64(lat)
        long = np.float64(long)
        target_coord = (lat,long)
        nearest_distances = self.find_nearest_postoffices_distances(lat,long)
        indices = nearest_distances.index.to_numpy()
        nearest_coordinates = self.df.loc[indices,['Latitude','Longitude']].to_numpy()
        origin = f"{lat},{long}"
        destinations = []
        for coords in nearest_coordinates:
            destinations.append(f"{coords[0]},{coords[1]}")
        destination_query = '|'.join(destinations)
        print(origin,destination_query)
        distance_matrix = self.client.routing.distance_matrix(
            origin,
            destination_query
        )['rows'][0]['elements']
        newdf = pd.DataFrame(distance_matrix).sort_values(by="distance")
        ser = newdf.iloc[0]
        idx = int(ser.name)    
        coords = nearest_coordinates[idx]
        print(coords)
        data = ser.to_dict()
        data['lat'] = float(coords[0])
        data['long'] = float(coords[1])
        return data
    
    def get_geocoded_and_destination_coordinates(self, address):
        data = self.get_geo_coded_data_with_coordinates(address)
        source_lat = data['response']['long']
        source_long = data['response']['lat']
        destination_data = self.get_nearest_postoffice(source_lat,source_long)
        dest_lat = destination_data['lat']
        dest_long = destination_data['long']
                
        return {
            'receiver_lat':source_lat,
            'receiver_long':source_long,
            'post_lat':dest_lat,
            'post_long':dest_long,
        }
        
    
    

In [18]:
mmi = MMI(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)


In [19]:
mmi.get_geocoded_and_destination_coordinates(address='Hindustan Lever Ltd., Express Building 1st Floor, Bahadur Shah Zafer Marg, P.O. Box. 7003,, NEW DELHI-110002')

28.633115,77.241392 28.6317778,77.24025|28.6317778,77.24025|28.629917,77.239895|28.6338611,77.2462222|28.6284722,77.2445555
[28.6284722 77.2445555]


{'source_lat': '28.633115',
 'source_long': '77.241392',
 'dest_lat': 28.6284722,
 'dest_long': 77.2445555}

In [9]:
mmi.get_geo_coded_data_with_coordinates('A 330, Moti Bagh 1, New Delhi, 110021')

{'status': 'success',
 'status_code': 200,
 'response': {'houseNumber': '330',
  'houseName': '',
  'poi': '',
  'street': '',
  'subSubLocality': 'Type 2',
  'subLocality': 'Block A',
  'locality': 'Moti Bagh North',
  'village': '',
  'subDistrict': 'Vasant Vihar',
  'district': 'New Delhi District',
  'city': 'New Delhi',
  'state': 'Delhi',
  'pincode': '110021',
  'formattedAddress': '330, Type 2, Block A, Moti Bagh North, Vasant Vihar, New Delhi District, New Delhi, Delhi, 110021',
  'eLoc': 'P6UNOV',
  'geocodeLevel': 'houseNumber',
  'confidenceScore': 0.8,
  'long': '28.581916',
  'lat': '77.176372'}}

# Server Setup

In [32]:
SUBDOMAIN = "postmitra"
PORT = 8001

In [29]:
try:
    print(TunneledServer.flag == True)
    if not TunneledServer.flag:
        raise Exception('')
except:
    class TunneledServer:
        flag = False
        tunnels = {}
        processes = {}
        def __init__(self, port, app):
            self.tunnel = multiprocessing.Process(target=TunneledServer.start_tunnel, args = (port,))
            self.local_server = multiprocessing.Process(target=app.run,kwargs = {'port' : port})
            self.port = port
        @staticmethod
        def start_tunnel(port):
            print(f"Started a Tunnel on port {port}")
            !lt --port {port} --subdomain {SUBDOMAIN}
            
        def start_self_tunnel(self,force = False):
            if self.port in TunneledServer.tunnels:
                if force:
                    TunneledServer.tunnels[self.port].terminate()
                    TunneledServer.tunnels[self.port] = self.tunnel
                    self.tunnel.start()
                    print(f"Restated the Tunnel on Port {self.port}")
                else:
                    self.tunnel = TunneledServer.tunnels[self.port]
                    print(f"Tunnel Already established on port {self.port}")
            else:
                self.tunnels[self.port] = self.tunnel
                self.tunnel.start()
        def start_local_server(self,force = False):
            if self.port in TunneledServer.processes:
                if force:
                    TunneledServer.processes[self.port].terminate()
                    TunneledServer.processes[self.port] = self.local_server
                    self.local_server.start()
                    print(f"Restated the Process on Port {self.port}")
                else:
                    self.local_server = TunneledServer.processes[self.port]
                    print(f"Process Already Running on port {self.port}")
            else:
                self.processes[self.port] = self.local_server
                self.local_server.start()
                
        def start(self,force_tunnel = False, force_process = False):
            self.start_self_tunnel(force_tunnel)
            self.start_local_server(force_process)
            
        def terminate_process(self):
            self.local_server.terminate()
            TunneledServer.processes.pop(self.port)
            print(f"Terminated Server Running on port {self.port}")
            
        def terminate_tunnel(self):
            self.tunnel.terminate()
            TunneledServer.tunnels.pop(self.port)
            print(f"Terminated Tunnel Established on port {self.port}")
            
        def terminate(self):
            self.terminate_process()
            self.terminate_tunnel()

In [25]:

import urllib.parse


class MMIServer:
    def __init__(self, mmi_client_id, mmi_client_secret, app_name = __name__):
        self.mmi = MMI(client_id=mmi_client_id, client_secret=mmi_client_secret)
        self.app = Flask(app_name)
        self.initialize_routes()
    
    def initialize_routes(self):
        @self.app.post('/api/v1/get-geocodes')
        def get_geocodes():
            address = urllib.parse.unquote(flask_request.args.get('address'))
            geocoded_data = self.mmi.get_geo_coded_data_with_coordinates(address=address)
            return geocoded_data
        @self.app.post('/api/v1/get-coords')
        def get_post_office_coords():
            address = urllib.parse.unquote(flask_request.args.get('address'))
            
            return jsonify(self.mmi.get_geocoded_and_destination_coordinates(
                address
            ))
        
        @self.app.post('/api/v1/parse-image')
        def parse_image_from_url():
            
        
            
    def get_app(self):
        return self.app
            
        

In [26]:
mmiServer = MMIServer(CLIENT_ID, CLIENT_SECRET)
app = mmiServer.get_app()

In [30]:

url = 'https://loca.lt/mytunnelpassword'
response = requests.get(url)

# Check if the request was successful
for i in range(3):
    if response.status_code == 200:
        # Get the content of the response
        content = response.text
        print(f'Max tunnel password {content}')
        break
    else:
        print(f"Failed to retrieve the content Retrying after two seconds. Status code: {response.status_code}")
        time.sleep(10)
else:
    raise Exception("Unable to fetch the tunnel password")

Max tunnel password 34.141.172.126


In [33]:
web_server = TunneledServer(app=app, port=PORT)

In [ ]:
web_server.start()

Started a Tunnel on port 8001


 * Serving Flask app '__main__'
 * Debug mode: off


your url is: https://clever-otter-10.loca.lt
28.581663,77.175866 28.5807778,77.1727222|28.5722222,77.1732778|28.5706389,77.1755|28.5787222,77.1648889|28.5701944,77.1713056
[28.5807778 77.1727222]


In [ ]:
web_server.terminate()

Terminated Tunnel Established on port 8001


In [7]:
df

,CircleName,RegionName,DivisionName,OfficeName,Pincode,OfficeType,Delivery,District,StateName,Latitude,Longitude
0,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Peddakotla B.O,515631,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.5689,77.85624
1,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Pinnadhari B.O,515631,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.5281,77.857014
2,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Yerraguntapalle B.O,515631,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.561111,77.85715
3,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Obulareddipalli B.O,515581,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.2488,78.2588
4,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Odulapalli B.O,515581,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.24555,78.2477
...,...,...,...,...,...,...,...,...,...,...,...
157121,Uttar Pradesh Circle,Bareilly Region,Meerut Division,Rly Road Meerut SO,250002,PO,Non Delivery,MEERUT,UTTAR PRADESH,28.98,77.6800000
157122,Uttar Pradesh Circle,Bareilly Region,Meerut Division,SGMandi SO,250002,PO,Non Delivery,MEERUT,UTTAR PRADESH,28.9724,77.6753600
157123,Uttar Pradesh Circle,Bareilly Region,Meerut Division,W K Road SO,250001,PO,Non Delivery,MEERUT,UTTAR PRADESH,28.99,77.7100000
157124,Kerala Circle,Calicut Region,Calicut Division,Kakkoti SO,673611,PO,Delivery,KOZHIKODE,KERALA,11.24529,75.7784550


In [70]:
def normalize_coordinates(coordinate):
    coordinate = str(coordinate)
    split_text = [i for i in coordinate if (i >= '0' and i <= '9') or i == '.']
    split_text = ''.join(split_text)
    if not split_text:
        return np.nan
    
    return np.float64(split_text)

In [92]:
df

,CircleName,RegionName,DivisionName,OfficeName,Pincode,OfficeType,Delivery,District,StateName,Latitude,Longitude
0,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Peddakotla B.O,515631,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.5689,77.85624
1,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Pinnadhari B.O,515631,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.5281,77.857014
2,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Yerraguntapalle B.O,515631,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.561111,77.85715
3,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Obulareddipalli B.O,515581,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.2488,78.2588
4,Andhra Pradesh Circle,Kurnool Region,Hindupur Division,Odulapalli B.O,515581,BO,Delivery,ANANTAPUR,ANDHRA PRADESH,14.24555,78.2477
...,...,...,...,...,...,...,...,...,...,...,...
157121,Uttar Pradesh Circle,Bareilly Region,Meerut Division,Rly Road Meerut SO,250002,PO,Non Delivery,MEERUT,UTTAR PRADESH,28.98,77.68
157122,Uttar Pradesh Circle,Bareilly Region,Meerut Division,SGMandi SO,250002,PO,Non Delivery,MEERUT,UTTAR PRADESH,28.9724,77.67536
157123,Uttar Pradesh Circle,Bareilly Region,Meerut Division,W K Road SO,250001,PO,Non Delivery,MEERUT,UTTAR PRADESH,28.99,77.71
157124,Kerala Circle,Calicut Region,Calicut Division,Kakkoti SO,673611,PO,Delivery,KOZHIKODE,KERALA,11.24529,75.778455


In [200]:
class temp:
    def __init__(self):
        
        
    

In [201]:
t = temp()

In [202]:
t.get_nearest_postoffice(28,77)

[28.0405833 77.0405833]


{'duration': 268,
 'distance': 4480,
 'polyline': 's~{iDav`uM??kT^cy@xF}EVeAZgBt@uEfBkPlEeBNcA@aNcAeSmBsj@cFQAPm@JOz@_CzE{KvAeCfIuMjBgDj@iA',
 'status': 'OK',
 'lat': 28.0405833,
 'long': 77.0405833}

In [182]:
ser

duration                                                  268
distance                                                 4480
polyline    s~{iDav`uM??kT^cy@xF}EVeAZgBt@uEfBkPlEeBNcA@aN...
status                                                     OK
lat                                                         2
long                                                        8
Name: 0, dtype: object

In [127]:
distance_matrix = client.routing.distance_matrix(
    "28.71866756826579,77.03699668376802",
    "28.638555357785652,76.96550156007675|28.53966907108812,77.05190669909288"
)
pprint(distance_matrix)

{'rows': [{'elements': [{'distance': 21473,
                         'duration': 3521,
                         'polyline': 'sbhnDypeuMQ?sDCsA@cB@uLAy@?kA?oACsCAeAAkC?Y?c@AgA?uBCYfDKz@SrBKrAG~@G`@IPINSRQNaCrB_@Zm@}@iJfAqBX_B\\@nA@P?nCErCMzAELVFp@Dt@Bp@Dr@BP@J@hDPPB\\HXLPLj@X`Bx@vBbA`At@bBdAn@Z`A^~CnA`Bj@rBl@h@TVP^V|BxAfBjAr@VXDPATO\\a@JKPI`@GzEH`OB\\?xA?xE@lA@b@?xEDjB@H?fABH?L?\\?z@?dCBhB@jA@r@?J?h@?fA@v@BxEBhA?`C@bA?jB@|@?L@fB?fB?p@?rEDtABfB@tA?b@@T?nABT?T?|RLvFBR?xIDv@ERETOZ_@RGZI|HTZCb@GREB?\\IVB`@DfAJ`CLtDPt@Ph@\\Zf@Tj@TfAj@nCLh@Rb@b@V~Ap@B@`@JJ?R?ZEf@KjAWjA_@hAa@PEp@[VM^Q@?HE\\St@k@lBgATOt@a@pAi@l@StCy@pCw@^AT?^AZ@A`@GlBK~CQjFMdEAPUhHOfE]bKm@|FEhA_@tJGdJQlE]`JM`DSxEIlBKtC]bIEnAIbBKnCc@lKQlEEfAQpEAPX@b@Bv@J\\D`ELL@H@PBTF~Bn@z@X|@\\\\HlFrAxBj@p@NRFN@zCZr@Fv@DT?nE@D@`@?tFJx@@PALAz@[JCD?D?H@h@L`Bd@~@RD@B@BDBB@BDZNz@?Bf@fFBFJj@DJBFDD@@DBHBb@FT?N?\\EFCDCDCBENUt@sADSNk@VgAFSJq@bCr@|F`BzCz@jG~A|FxAhN|DhOfE`Bj@bDjAHDF@`Ct@dAV\\FdHb@nBNRFl@Vt@`@zA`A`IxEvAz@b@LXDV@n@@~O?zC?nKCX@\\BXHVJPJVN^Z

In [121]:
t = temp()

In [122]:
t.find_nearest_postoffices(28,77)

89658     0.028018
37653     0.028284
117368    0.038534
155016    0.098521
37656     0.169706
dtype: float64